In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import os

from sklearn.metrics import f1_score
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

SEED = 53

# 1.  Loading the data from Notebook

In [2]:
class PATH:
        TRAIN = 'train'
        VALID = 'valid'
        TEST = 'test'

dataset = { "train_data" : PATH.TRAIN,
            "valid_data" : PATH.VALID,
            "test_data" : PATH.TEST }

datalist = []

for path in dataset.values():
    data = {"imgpaths": [] , "labels": []}
    paths = os.listdir(path)

    for folders in paths:
        paths_folders = os.path.join(path, folders)
        filelist = os.listdir(paths_folders)
        for files in filelist:
            paths_folders_files = os.path.join(paths_folders, files)
            data["imgpaths"].append(paths_folders_files)
            data["labels"].append(folders)
        
    datalist.append(data.copy())
    data.clear()

train_df = pd.DataFrame(datalist[0] , index=range(len(datalist[0]['imgpaths'])))
valid_df = pd.DataFrame(datalist[1] , index=range(len(datalist[1]['imgpaths'])))
test_df = pd.DataFrame(datalist[2] , index=range(len(datalist[2]['imgpaths'])))

# 2.  Data preprocessing

In [3]:
BatchSiz = 20
ImgSiz = (224, 224)

# Data Augumentation
train_generator = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.10,
    horizontal_flip=True,
    validation_split=0.1
)

generator = ImageDataGenerator()

# Dataframe --> Preprocessed Image data

train_images = train_generator.flow_from_dataframe(
    dataframe = train_df,
    x_col = 'imgpaths',
    y_col = 'labels',
    target_size = ImgSiz,
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = BatchSiz,
    shuffle = True,
    seed = SEED
)

valid_images = generator.flow_from_dataframe(
    dataframe = valid_df,
    x_col = 'imgpaths',
    y_col = 'labels',
    target_size = ImgSiz,
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = BatchSiz,
    shuffle = False,
    seed = SEED
)

test_images = generator.flow_from_dataframe(
    dataframe = test_df,
    x_col = 'imgpaths',
    y_col = 'labels',
    target_size = ImgSiz,
    color_mode = 'rgb',
    class_mode = 'categorical',
    batch_size = BatchSiz,
    shuffle = False,
    seed = SEED
)

Found 84635 validated image filenames belonging to 525 classes.
Found 2625 validated image filenames belonging to 525 classes.
Found 2625 validated image filenames belonging to 525 classes.


# 3.  Fine Tuning

In [4]:
# When loading a whole model using 'load_model'

def F1_score(y_true, y_pred):
    # 將預測值轉換為整數類別
    y_pred = tf.argmax(y_pred, axis=-1)
    y_true = tf.argmax(y_true, axis=-1)
    
    # 計算 TP, FP, FN
    TP = tf.reduce_sum(tf.cast(y_true * y_pred, 'float32'))
    FP = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float32'))
    FN = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float32'))
    
    # 計算 precision 和 recall
    precision = TP / (TP + FP + tf.keras.backend.epsilon())
    recall = TP / (TP + FN + tf.keras.backend.epsilon())
    
    # 計算 F1 score
    f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
    return f1

model = tf.keras.models.load_model('EfficientNetB0-525-(224 X 224)- 98.97.h5', custom_objects={'F1_score': F1_score})

In [5]:
# Unfreeze the EfficientNetB0
model.trainable = True

# Compile for finetuning model
model.compile(
    optimizer = Adam(learning_rate = 1e-5),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

# Callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint('EfficientNetB0_finetuned.h5', save_best_only=True)
earlystopping_cb = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=3, restore_best_weights=True)
reducelr_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=3, mode='auto')

# Train the finetuning model
history = model.fit(
    train_images,
    steps_per_epoch=len(train_images),
    validation_data=valid_images,
    validation_steps=len(valid_images),
    epochs=1, #20
    callbacks=[checkpoint_cb, earlystopping_cb, reducelr_cb]
)

model.save_weights('./best-model/EfficientNetB0_weights')

4232/4232 [==============================] - 1541s 361ms/step - loss: 0.2629 - accuracy: 0.9882 - val_loss: 0.3382 - val_accuracy: 0.9741 - lr: 1.0000e-05


# 4.  Evaluation Index

## Test Score

In [6]:
# Test score
res = model.evaluate(test_images, verbose=0)
print('Test Loss =', res[0]*100)
print('Test Accuracy =', res[1]*100)

Test Loss = 34.20040309429169
Test Accuracy = 97.18095064163208


## F1 Score

> F1 score is a metric used to evaluate classification models. 

> It balances precision (ability to avoid false positives) and recall (ability to find all positives) into a single score. 
> It ranges from 0 to 1, with higher values indicating better model performance.
> 

In [7]:
y_test = test_images.classes
y_pred = np.argmax(model.predict(test_images), axis = 1)

132/132 [==============================] - 8s 51ms/step


In [8]:
# Compute F1 Score (A value related to Recall and Precision)
f1 = f1_score(y_test, y_pred, average='macro')
print("F1 Score:", f1)

F1 Score: 0.9711201814058957
